In [1]:
!pip install trl

  Using cached trl-0.23.0-py3-none-any.whl.metadata (11 kB)
  Using cached datasets-4.1.1-py3-none-any.whl.metadata (18 kB)
  Using cached transformers-4.56.2-py3-none-any.whl.metadata (40 kB)
  Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached trl-0.23.0-py3-none-any.whl (564 kB)
Using cached datasets-4.1.1-py3-none-any.whl (503 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (42.8 MB)
Using cached transformers-4.56.2-py3-none-any.whl (11.6 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstall

In [2]:
!pip install -U bitsandbytes

  Using cached bitsandbytes-0.47.0-py3-none-manylinux_2_24_x86_64.whl.metadata (11 kB)
Using cached bitsandbytes-0.47.0-py3-none-manylinux_2_24_x86_64.whl (61.3 MB)


In [3]:
!pip install torch

In [4]:
!pip install peft

  Using cached peft-0.17.1-py3-none-any.whl.metadata (14 kB)
Using cached peft-0.17.1-py3-none-any.whl (504 kB)


In [5]:
from trl import SFTTrainer

2025-09-24 14:12:27.309782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758723147.333988    1293 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758723147.347380    1293 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-24 14:12:27.557495: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import pandas as pd

In [7]:
from datasets import load_dataset
from datasets import Dataset
import json

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./local_model_dir", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    "./local_model_dir",
    trust_remote_code=True,
)

# Get the device from the model (for quantized models, this is usually 'cuda' if available)
#device = next(model.parameters()).device

In [9]:
train_df = pd.read_csv("./data/train_dataset.csv")

In [10]:
train_df = Dataset.from_pandas(train_df)

In [11]:
train_df[5]

{'tools': '[{"name": "chi_square_independence_test", "description": "Performs a Chi-Square test for independence on a 2x2 contingency table.", "parameters": {"contingency_table": {"description": "A 2x2 contingency table represented as a list of lists.", "type": "List[List[int]]", "default": 0.05}, "significance_level": {"description": "The significance level for the Chi-Square test. Defaults to 0.05.", "type": "float, optional", "default": 0.05}}}]',
 'answers': '[{"name": "chi_square_independence_test", "arguments": {"contingency_table": [[80, 20], [60, 40]], "significance_level": 0.05}}]',
 'query': 'A psychologist wants to test if there is a significant relationship between a personality trait and a behavior. They have the following data: [[80, 20], [60, 40]]. Perform a Chi-Square test for independence with a significance level of 0.05.',
 'id': 52914}

In [12]:
train_df

Dataset({
    features: ['tools', 'answers', 'query', 'id'],
    num_rows: 489
})

In [13]:
# Select the first 3 rows for testing
small_sample = train_df.select(range(3))

In [14]:
small_sample

Dataset({
    features: ['tools', 'answers', 'query', 'id'],
    num_rows: 3
})

In [15]:
SYSTEM_PROMPT = "You are a helpful assistant that uses the provided tools to answer."

In [24]:
def format_dataset(example):
    query = example['query']
    tools = json.loads(example['tools'])
    expected_calls = json.loads(example['answers'])
                               
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": query}, 
        {"role": "assistant", "tool_calls": expected_calls}
    ]
    
    texts = tokenizer.apply_chat_template(messages,tools = tools, tokenize=False, add_generation_prompt=False)
    #print(texts)
    return {"text": texts}

In [18]:
examples = small_sample.map(format_dataset, remove_columns=train_df.column_names) 

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

<|im_start|>system
You are a helpful assistant that uses the provided tools to answer.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"name": "circle_area", "description": "Computes the area of a circle given its radius.", "parameters": {"radius": {"description": "The radius of the circle.", "type": "float"}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
Calculate the area of circles with radii 3.14 and 6.28. Oh, and don't forget one with radius 9.42, please.<|im_end|>
<|im_start|>assistant
<think>

</think>

<tool_call>
{"name": "circle_area", "arguments": {"radius": 3.14}}
</tool_call>
<tool_call>
{"name": "circle_area", "arguments": {"radius": 6.28}}
</tool_call>
<tool_call>
{"na

In [20]:
processed_train = train_df.map(format_dataset, remove_columns=train_df.column_names) 

Map:   0%|          | 0/489 [00:00<?, ? examples/s]

In [21]:
processed_train[5]['text']

'<|im_start|>system\nYou are a helpful assistant that uses the provided tools to answer.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"name": "chi_square_independence_test", "description": "Performs a Chi-Square test for independence on a 2x2 contingency table.", "parameters": {"contingency_table": {"description": "A 2x2 contingency table represented as a list of lists.", "type": "List[List[int]]", "default": 0.05}, "significance_level": {"description": "The significance level for the Chi-Square test. Defaults to 0.05.", "type": "float, optional", "default": 0.05}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call><|im_end|>\n<|im_start|>user\nA psychologist wants to test if there is a significant 

In [29]:
from peft import LoraConfig
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "v_proj",
        "o_proj",
        "k_proj",
        "up_proj",
        "down_proj",
        "gate_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

In [30]:
model.add_adapter(lora_config, adapter_name="lora_1")

In [31]:
model.gradient_checkpointing_enable()


In [32]:
from trl import SFTConfig

training_args = SFTConfig(
    output_dir="./qwen3-function-calling-ft",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=True,
    bf16=False,
    report_to="none"
)

# ✅ SFTTrainer Configuration
trainer = SFTTrainer(
    model=model,
    train_dataset=processed_train,
    formatting_func=None,  # optional: custom formatting for messages
    args=training_args
)

# ✅ Start Training
trainer.train()

# ✅ Save Model
trainer.save_model("./qwen3-function-calling-ft")
tokenizer.save_pretrained("./qwen3-function-calling-ft")

Adding EOS to train dataset:   0%|          | 0/489 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/489 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/489 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,2.251700
20,1.199200
30,0.696600
40,0.494600
50,0.419800
60,0.360400
70,0.316900
80,0.270300
90,0.251500
100,0.221100


('./qwen3-function-calling-ft/tokenizer_config.json',
 './qwen3-function-calling-ft/special_tokens_map.json',
 './qwen3-function-calling-ft/chat_template.jinja',
 './qwen3-function-calling-ft/vocab.json',
 './qwen3-function-calling-ft/merges.txt',
 './qwen3-function-calling-ft/added_tokens.json')

In [1]:
processed_train[0]

NameError: name 'processed_train' is not defined

In [ ]:
torch.cuda.memory._record_memory_history()
torch.cuda.memory._dump_snapshot("snapshot.pickle")

In [ ]:
print(torch.cuda.memory._record_memory_history())

None


In [33]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (lora_1): Identity()
            )
            (lora_A): ModuleDict(
              (lora_1): Linear(in_features=2560, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (lora_1): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (lora_1):

In [27]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (lora_1): Identity()
            )
            (lora_A): ModuleDict(
              (lora_1): Linear(in_features=2560, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (lora_1): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (lora_1):

In [28]:
trainer.save_model('./finetuned_qwen4b')